# 첫 번째 전자 상거래 추천 시스템 구축하기

이 노트북에서는 도메인 데이터 세트 그룹을 구축하는 단계와 가상 소매점 데이터 세트에 대해 생성된 데이터를 기반으로 제품 추천을 반환하는 추천 시스템을 안내합니다. 목표는 특정 사용자를 기준으로 연관성 높은 제품을 추천하는 것입니다.

이 합성 데이터는 [Retail Demo Store 프로젝트](https://github.com/aws-samples/retail-demo-store)에서 가져온 것입니다. 데이터와 잠재적인 사용 사례에 대해 자세히 알아보려면 링크를 클릭하세요.

# 이 노트북을 사용하는 방법

코드는 아래와 같이 셀로 구분됩니다. 이 페이지 맨 위에 삼각형 모양의 실행(Run) 버튼이 있습니다. 이 버튼을 클릭하여 각 셀을 실행하고 다음 셀로 이동하거나, 셀에 있는 동안 `Shift` + `Enter`를 눌러 셀을 실행한 후 다음 셀로 이동할 수 있습니다.

셀이 실행되면, 셀이 실행되는 동안에는 옆의 라인에 `*`가 표시되며, 셀 내의 모든 코드 실행을 완료하고 나면 이 기호가 실행을 완료한 마지막 셀을 나타내는 숫자로 업데이트됩니다.

아래의 지침에 따라 셀을 실행하여, 사례에 최적화된 추천을 사용해 Amazon Personalize를 시작하세요.

## 가져오기
Python은 다양한 라이브러리 컬렉션과 함께 제공되며, 핵심 데이터 과학 도구로서 설치된 [boto3](https://aws.amazon.com/sdk-for-python/)(python용 AWS SDK), [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/) 등을 라이브러리와 함께 가져와야 합니다.

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime

그런 다음 사용자 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 검증해야 합니다. 아래 코드 줄은 이 작업을 수행합니다.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## S3 버킷 및 데이터 출력 위치 지정

Amazon Personalize는 데이터의 소스 역할을 할 S3 버킷이 필요합니다. 아래의 코드는 고유한 `bucket_name`을 사용하여 버킷을 만듭니다.

Amazon S3 버킷은 Amazon Personalize 리소스와 동일한 리전에 있어야 합니다. 

In [ ]:
# Sets the same region as current Amazon SageMaker Notebook
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print('region:', region)

# Or you can specify the region where your bucket and model will be domiciled
# region = "us-east-1" 

s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizemanagedretailers"
print('bucket_name:', bucket_name)

try: 
    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket = bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
    
except Exception as e:
    print (e)
    print("Bucket already exists. Using bucket", bucket_name)

## 훈련 데이터 다운로드, 준비 및 업로드

먼저 데이터(훈련 데이터)를 다운로드해야 합니다. 이 튜토리얼에서는 소매점 데이터 세트의 구매 내역을 사용합니다. 이 데이터 세트에는 사용자 ID, 항목 ID, 고객과 항목 간의 상호 작용 및 이 상호 작용이 발생한 시간(타임스탬프)이 포함되어 있습니다.

### 데이터 세트 다운로드 및 탐색

In [ ]:
!aws s3 cp s3://retail-demo-store-us-east-1/csvs/items.csv .
!aws s3 cp s3://retail-demo-store-us-east-1/csvs/interactions.csv .

데이터 세트가 Electronics_Store_puchase_history.csv로 다운로드되었습니다.

데이터 세트의 특성을 통해 데이터 세트에 대해 자세히 알아보겠습니다.

In [ ]:
df = pd.read_csv('./interactions.csv')
df

In [ ]:
df.EVENT_TYPE.value_counts()

In [ ]:
def convert_event_type(event_type_in_some_format):
    if(event_type_in_some_format == "ProductViewed"):
        return "View"
    if(event_type_in_some_format == "OrderCompleted"):
        return "Purchase"
    else:
        return event_type_in_some_format

df['EVENT_TYPE'] = df['EVENT_TYPE'].apply(convert_event_type)

In [ ]:
df.EVENT_TYPE.value_counts()

ECOMMERCE 추천에서는 상호 작용의 컨텍스트를 이해하기 위해 특정 EVENT_TYPE 값을 제공해야 하므로 상호 작용 EVENTYPE 열을 수정하려고 합니다.

In [ ]:
df.info()

위의 2개의 셀에서 데이터에 9개의 열, 675004개의 행이 있으며 헤더는 다음과 같습니다. ITEM_ID, USER_ID, EVENT_TYPE, TIMESTAMP 및 DISCOUNT.

Amazon Personalize 상호 작용 스키마와 호환되려면 이 데이터 세트에서 Amazon Personalize 기본 열 이름과 호환되는 열 머리글이 필요합니다(열 이름에 대한 자세한 내용은 [여기](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html) 참조).

## 상호 작용 데이터 준비


### 열 삭제

이 데이터 세트의 일부 열은 모델에 가치를 더해주지 않으므로 이 데이터 세트에서 삭제해야 합니다. 할인과 같은 열이 이에 해당합니다.

In [ ]:
test=df.drop(columns=['DISCOUNT'])
df=test
df.sample(10)

아래의 셀에서 정리된 데이터를 "final_training_data.csv 파일에 기록합니다.

In [ ]:
df.to_csv("cleaned_training_data.csv")

### S3에 업로드
이제 훈련 데이터를 Amazon Personalize에 사용할 준비가 되었으므로, 다음 단계는 앞서 만든 S3 버킷에 데이터를 업로드하는 것입니다.

In [ ]:
interactions_file_path = 'cleaned_training_data.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_file_path).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_file_path


## S3 버킷 및 IAM 역할 구성

지금까지 이 Jupyter 노트북을 실행하는 인스턴스에 연결된 Amazon EBS 인스턴스에 데이터를 다운로드하고 조작하고 저장했습니다. 하지만 Amazon Personalize에는 데이터의 소스 역할을 하는 S3 버킷과 해당 버킷에 액세스하기 위한 IAM 역할이 필요합니다. 이 버킷과 역할을 설정해보겠습니다.


## S3 버킷 정책 설정
Amazon Personalize가 S3 버킷의 콘텐츠를 읽을 수 있어야 합니다. 이를 허용하는 버킷 정책을 추가합니다.

참고: 이 노트북에서 코드를 실행하는 데 사용 중인 역할에 S3 버킷 정책을 수정하는 데 필요한 권한이 있는지 확인합니다.

In [ ]:
s3 = boto3.client("s3")
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

## 데이터 세트 그룹 생성 및 생성이 완료될 때까지 기다리기
Personalize에서 가장 큰 그룹은 데이터 세트 그룹으로, 데이터, 이벤트 트래커, 솔루션, 추천 시스템 및 캠페인을 격리합니다. 공통적인 데이터 컬렉션을 공유하는 것들끼리 한데 그룹화합니다. 원하는 경우 아래의 이름을 자유롭게 변경해도 됩니다.

### 데이터 세트 그룹 생성

In [ ]:
response = personalize.create_dataset_group(
    name='personalize_ecomemerce_ds_group',
    domain='ECOMMERCE'
)

dataset_group_arn = response['datasetGroupArn']
print(json.dumps(response, indent=2))

데이터 세트 그룹이 활성 상태가 될 때까지 기다리기
아래의 항목에서 데이터 세트 그룹을 사용하려면 먼저 아래의 셀을 실행하고 활성 상태로 표시될 때까지 기다립니다.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## 상호 작용 스키마 생성
Personalize가 데이터를 이해하기 위한 핵심 구성 요소는 아래에 정의된 스키마에서 얻어집니다. 이 구성은 Personalize 서비스에 CSV 파일을 통해 제공된 데이터를 수집하는 방법을 알려줍니다. 열과 유형은 위에서 만든 파일에 있는 내용과 일치합니다.

In [ ]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
            
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-ecommerce-interatn_group",
    domain = "ECOMMERCE",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

## 데이터 세트 생성
그룹 다음으로는 실제 데이터 세트를 생성해야 합니다.

### 상호 작용 데이터 세트 생성

In [ ]:
dataset_type = "INTERACTIONS"

create_dataset_response = personalize.create_dataset(
    name = "personalize_ecommerce_demo_interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Personalize 역할 생성
또한 Amazon Personalize가 특정 작업을 실행할 수 있는 권한을 가지려면, AWS에서 역할을 수임할 수 있어야 합니다. 아래의 코드 줄은 이 권한을 부여합니다.

참고: 이 노트북에서 코드를 실행하는 데 사용 중인 역할에 새 역할을 생성하는 데 필요한 권한이 있는지 확인합니다.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleEcommerceDemoRecommender"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)


## 데이터 가져오기
앞서 데이터 세트 그룹과 데이터 세트를 만들어 정보를 저장했으므로, 이제 모델 구축 사용 사례를 위해 S3에서 Amazon Personalize로 데이터를 로드하는 가져오기 작업을 실행합니다.
### 상호 작용 데이터 세트 가져오기 작업 생성

In [ ]:
create_interactions_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize_ecommerce_demo_interactions_import",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_file_path)
    },
    roleArn = role_arn
)

dataset_interactions_import_job_arn = create_interactions_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interactions_dataset_import_job_response, indent=2))

데이터 세트 가져오기 작업이 활성 상태가 될 때까지 기다리기
가져오기 작업이 완료되는 데 시간이 걸릴 수 있습니다. 아래에 활성 상태로 표시될 때까지 기다리세요.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_interactions_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## 추천 시스템 사용 사례 선택

각 도메인마다 사용 사례가 다릅니다. 추천 시스템을 만들 때에는 특정 사용 사례에 맞게 만들어야 하며, 사용 사례마다 추천을 받는 데 있어서 요구 사항이 다릅니다.


In [ ]:
available_recipes = personalize.list_recipes(domain='ECOMMERCE') # See a list of recommenders for the domain. 
display (available_recipes['recipes'])

여기서는 "Customers who viewed X also viewed (X를 검색한 고객들이 함께 본 상품)" 유형의 추천 시스템을 만듭니다. 이 추천 시스템에서는 사용자가 지정한 항목을 기준으로 고객이 시청한 다른 항목에 대한 추천을 제공합니다. 이 사용 사례에서, Amazon Personalize는 `Purchase` 이벤트에 지정한 userId를 기준으로 사용자가 구매한 항목을 자동으로 필터링합니다.

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'viewed_x_also_viewed_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed',
  datasetGroupArn = dataset_group_arn
)
viewed_x_also_viewed_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

이제 "Recommended For You (고객님을 위한 추천 상품)" 유형의 두 번째 추천 시스템을 만들어 보겠습니다. 이 유형의 추천 시스템은 지정한 사용자를 기준으로 개인화된 항목 추천을 제공합니다. 이 사용 사례에서, Amazon Personalize는 `Purchase` 이벤트에 지정한 userId를 기준으로 사용자가 구매한 항목을 자동으로 필터링합니다.

[도메인별 추가 사용 사례](https://docs.aws.amazon.com/personalize/latest/dg/domain-use-cases.html)

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'recommended_for_you_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-recommended-for-you',
  datasetGroupArn = dataset_group_arn
)
recommended_for_you_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

추천 시스템 생성이 완료되고 `ACTIVE` 상태가 될 때까지 기다립니다. 추천 시스템의 상태를 주기적으로 확인합니다.

In [ ]:
%%time

max_time = time.time() + 10*60*60 # 10 hours

while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = viewed_x_also_viewed_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(viewed_x_also_viewed_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(viewed_x_also_viewed_arn))
        

    if status == "ACTIVE" or status == "CREATE FAILED":
        break
    else:
        print('The "Customers who viewed X also viewed" Recommender build is still in progress')
        
    time.sleep(60)
    
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommended_for_you_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(recommended_for_you_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(recommended_for_you_arn))
        break

    if status == "ACTIVE" or status == "CREATE FAILED":
        break
    else:
        print('The "Recommended for you" Recommender build is still in progress')
        
    time.sleep(60)

## 추천 시스템으로 추천 받기
이제 추천 시스템을 훈련했으므로 사용자를 위해 받을 수 있는 추천을 살펴보도록 하겠습니다!

In [ ]:
# reading the original data in order to have a dataframe that has both item_ids 
# and the corresponding titles to make out recommendations easier to read.
items_df = pd.read_csv('./items.csv')
items_df.sample(10)

In [ ]:
def get_item_by_id(item_id, item_df):
    """
    This takes in an item_id from a recommendation in string format,
    converts it to an int, and then does a lookup in a default or specified
    dataframe and returns the item description.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return items_df.loc[items_df["ITEM_ID"]==str(item_id)]['PRODUCT_DESCRIPTION'].values[0]
    except:
        print (item_id)
        return "Error obtaining item description"

"Customers who viewed X also viewed (X를 검색한 고객들이 함께 본 상품)" 추천 시스템을 사용하여 추천을 받아보겠습니다.

In [ ]:
# use a random valid id for a quick sanity check, modify the line of code bellow to a valid id in your dataset
get_item_by_id("c72257d4-430b-4eb7-9de3-28396e593381", items_df)

In [ ]:
# First pick a user
test_user_id = "777"

# Select a random item
test_item_id = "8fbe091c-f73c-4727-8fe7-d27eabd17bea" # a random item: 8fbe091c-f73c-4727-8fe7-d27eabd17bea

# Get recommendations for the user for this item
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = viewed_x_also_viewed_arn,
    itemId = test_item_id,
    userId = test_user_id,
    numResults = 10
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []

for item in item_list:
    item = get_item_by_id(item['itemId'], items_df)
    recommendation_list.append(item)

user_recommendations_df = pd.DataFrame(recommendation_list, columns = [get_item_by_id(test_item_id, items_df)])

pd.options.display.max_rows =10
display(user_recommendations_df)

"Recommended for you (고객님을 위한 추천 상품)"를 반환하는 추천 시스템에서 추천을 받습니다.

In [ ]:
# First pick a user
test_user_id = "777" 

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommended_for_you_arn,
    userId = test_user_id,
    numResults = 20
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    item = get_item_by_id(item['itemId'], items_df)
    recommendation_list.append(item)


user_recommendations_df = pd.DataFrame(recommendation_list, columns = [test_user_id])

pd.options.display.max_rows =20
display(user_recommendations_df)

## 검토
위의 코드를 사용하여 딥 러닝 모델을 성공적으로 훈련하여 이전 사용자 행동을 기반으로 한 항목 추천을 생성했습니다. 두 가지 기본 사용 사례에 대해 두 가지 추천을 생성했습니다.
이제 다음으로, 이 코드를 조정하여 다른 추천 시스템을 만들 수 있습니다.

## 다음 노트북에 대한 참고 사항:
다음 노트북에 필요한 몇 가지 값이 있습니다. 아래 셀을 실행하여 `Clean_Up_Resources.ipynb` 노트북에서 사용할 수 있도록 해당 값을 저장하세요.

이렇게 하면 해당 변수에 대해 저장된 데이터를 덮어쓰고, 변수가 이 노트북에 지정된 값으로 설정됩니다. 

In [ ]:
# store for cleanup
%store dataset_group_arn
%store role_name
%store region

`Building_Your_First_Recommender_Video_On_Demand.ipynb` 노트북을 실행한 경우 여기서 생성된 리소스로 `Clean_Up_Resources.ipynb`를 실행한 후 `Building_Your_First_Recommender_Video_On_Demand.ipynb` 노트북에서 이전 단계를 다시 실행하고 `Clean_Up_Resources.ipynb`를 다시 실행하여 해당 노트북에서 생성된 리소스를 제거하세요.